# Procesamiento de Datos con Python: Proyecto Final (Parte 02)

## Objetivo

Obtener un único dataframe a partir de los reportes de ventas previamente limpiados.

## Introducción

En la primera parte se realizó todo el proceso de limpieza de uno de los reportes de venta que entrega el sistema ASPEL-SAE 8.0, sin embargo, a fin de automatizar el proceso, se realizan los cambios para poder obtener los reportes y hacer un compendio de los reportes para su posterior uso.

## Procedimiento

### Obtención de datos.

A fin de automatizar el proceso, se recurre al uso de funciones, para ello los datasets están precargados dentro de una misma carpeta llamada `Datsets`.

Se crea entonces, la función que ha de extraer los datos, se utiliza como base los pasos explicados en la parte 01.

In [5]:
# Importación de bibliotecas necesarias
import pandas as pd
import numpy as np

'''
  Nombre: clean_csv
  Argumentos: file_name [cadena de texto]
  Retorno: df_final [dataframe]
  Función: Utilizada para leer el dataframe indicado desde un repositorio en 
           github y regresarlo limpio.
'''
def clean_csv(file_name):
  # Si la lectura y operaciones fueron correctas.
  try:
    # Creación de URL para la lectura de los CSV.
    url = 'https://raw.githubusercontent.com/LuisMaldonado1366/ProyectoFinal_Equipo03/main/Datasets/'+str(file_name)+'.csv'
    
    # Lectura de los archivos csv para guardarlos como dataframes
    df_src = pd.read_csv(url, encoding = 'unicode_escape', engine = 'python')
    
    # Copia del dataframe original.
    df = df_src
    
    # Eliminar primer columna que contiene los índices. 
    df.drop(df.columns[[0]], axis = 'columns', inplace = True)
    
    # Renombramiento de columnas.
    df.columns = ['sku',	'Unnamed:1', 'descripcion', 'Unnamed:2', 'Unnamed:3', 'cantidad', 'Unnamed:4',
                  'costo_total', 'Unnamed:5', 'precio_total', 'utilidad',	'porcentaje_utilidad',	'Unnamed:6',
                  'margen','Unnamed:7']
    
    # Eliminar productos ajenos al catálogo.
    df_sku = df[df['sku'].str.contains('AR') == True]
    
    # Reasignar SKU como índice, eliminar columna "producto" y reordenar por SKU
    df_sku = df_sku.set_index('sku', drop = True)
    df_sku.sort_index(axis = 'index', inplace = True)
    
    # Eliminación de columnas con únicamente valores NaN.
    df_dropped = df_sku.dropna(axis = 'columns', how = 'all')
    
    # Selección de primeras cuatro columnas.
    df_final = df_dropped.iloc[:,[0,1,2,3]]
    
    # Quitar comas del dataframe
    df_final = df_final.replace(',','', regex = True)
    
    # Diccionario de columnas y tipo de dato asociado.
    diccionario_de_conversion = {
        'cantidad': 'int',
        'costo_total': 'float',
        'precio_total': 'float'
    }
    # Casting de variables
    df_final = df_final.astype(diccionario_de_conversion)
    
    # Calculo de utilidad para agregar al dataframe.
    utilidad = pd.Series(df_final['precio_total']-df_final['costo_total'])
    df_final['utilidad'] = utilidad
    df_final['utilidad'].apply(np.round)

    # Retorno de dataframe procesado.
    return df_final

  # Si hubo un error.
  except:
    # Retorno de dataframe vacío.
    return pd.DataFrame()
    

Se comprueba el funcionamiento de la función pasando una cadena de texto que va indicada por el año a 4 dígitos (2021) seguido de un guión con el número de mes en formato de números naturales (-1).

In [6]:
# Comprobación de funcionamiento de la función.
df = clean_csv('2022-1')
df

,descripcion,cantidad,costo_total,precio_total,utilidad
sku,,,,,
AR0001,UNO R3 con cable USB compatible con,376,43632.40,66008.06,22375.66
AR0002,Cables Dupont Largos h-h,49,355.12,1014.67,659.55
AR0003,Cables Dupont Largos h-m,461,2920.46,9859.69,6939.23
AR0004,Cables Dupont Largos m-m,268,1611.26,7626.95,6015.69
AR0005,Protoboard 830 Pts MB-102,586,10182.14,17644.06,7461.92
...,...,...,...,...,...
AR3029,ATTINY10-TSHR MCU Microcontrolador,7,80.64,181.02,100.38
AR3043,XY-C100L Amplificador 100W,20,2758.80,4742.27,1983.47
AR3044,MBR20100CTG Diodo Schottky,2,13.78,31.04,17.26


In [7]:
# Comprobación del tipo de dato del dataframe
df.dtypes

descripcion      object
cantidad          int64
costo_total     float64
precio_total    float64
utilidad        float64
dtype: object

La empresa maneja un archivo de google sheets para consulta de todas las áreas donde se pueden apreciar los productos existentes asociados a las ubicaciones de almacén y descripción del producto. 

Se guarda una copia de este archivo como csv para poder utilizar en operaciones posteriores ya que los reporte que entrega ASPEL solo muestran aquellos productos que tuvieron movimiento en el mes, por ende, si un producto no tuvo ventas no ha de aparecer en el listado. A fin de poder obtener el registro de todos los productos se utiliza este inventario como referencia.

In [8]:
# Lectura del inventario.
df_inventory = pd.read_csv('https://raw.githubusercontent.com/LuisMaldonado1366/ProyectoFinal_Equipo03/main/Datasets/inventario.csv')
df_inventory

,SKU,Código Álmacen,DESCRIPCIÓN
0,AR0001,DES-G42-5X,UNO R3 con cable USB compatible con Arduino
1,AR0002,COM-C63-2XY,Cables Dupont Largos 20cm h-h
2,AR0003,COM-C63-3XY,Cables Dupont Largos 20cm h-m
3,AR0004,COM-C63-4XY,Cables Dupont Largos 20cm m-m
4,AR0005,EQ-FA82-2XY,Protoboard 830 Pts MB-102
...,...,...,...
3556,AR3652,-,NaN
3557,AR3653,-,NaN
3558,AR3654,-,NaN
3559,AR3655,-,NaN


Para el análisis no son necesarios las columnas de descripción y código de almacén, utilizar el SKU como índice y eliminar todo registro que no tenga descripción ya que son SKUs no asociados a algún producto aún.

In [54]:
# Eliminar registros nulos de inventario (SKUs no asociados a un producto).
df_inventory = df_inventory.dropna(axis = 'rows', how ='any')

# Seleccionar una columna para obtener un dataframe con SKUs únicamente. 
df_sku = pd.DataFrame(df_inventory['SKU'])

# Renombrar columnas.
df_sku.columns = ['sku']

# Establecer columna de SKU como índice y borrarla.
df_sku = df_sku.set_index('sku', drop = True)

# Ordenar por índice.
df_sku.sort_index(axis = 'index', inplace = True)

df_sku

""
sku
AR0001
AR0002
AR0003
AR0004
AR0005
...
AR3364
AR3365
AR3366


Con el propósito de facilitar el análisis a futuro, se dividirán los datos por cantidad, monto de venta y utilidad, por lo que se crean copias del inventario que se usarán en operaciones futuras y se les aplica una unión izquiera para preservar todos los SKUs adjuntando los datos de cada mes y renombrándolos.

In [55]:
# Copiar inventario para cada dataframe a utilizar.
df_quantity = df_sku
df_utility = df_sku
df_sales = df_sku

'''
  Nombre: merge_data
  Argumentos: data_frame_fixed [dataframe]. Dataframe al que se le hará la unión.
              dataframe_to_merge [dataframe]. Dataframe a unir.
              column_to_merge [cadena de texto]. Nombre de la columna a unir del segundo dataframe.
              new_column_name [cadena de texto]. Nombre para renombrar columna agregada.
  Retorno: dataframe_merged [dataframe].
  Función: Unir dos dataframes por el índice conservando todos los registros del dataframe izquierdo, 
           renombrando la nueva columna por el nombre indicado.
'''
def merge_data(data_frame_fixed, dataframe_to_merge, column_to_merge, new_colum_name):
  # Copiar el dataframe derecho.
  dataframe = dataframe_to_merge
  # Seleccionar únicamente la columna indicada.
  dataframe_temp = dataframe[column_to_merge]
  # Realizar unión izquierda de los dataframes.
  dataframe_merged = data_frame_fixed.merge(dataframe_temp.to_frame(), how = 'left', left_index = True, right_index = True)
  # Renombramiento de columna recién asignada.
  dataframe_merged.rename(columns = {column_to_merge : new_colum_name}, inplace = True)
  # Retorno de función.
  return dataframe_merged

# Realizar iteración por año.
for i in range(2021,2023):
  # Realizar iteración por mes.
  for j in range(1,13):
    # Creación de cadena para función clean_csv, "YYYY-MM".
    file_name = str(i) + '-' + str(j)
    # Lectura del dataframe.
    df_temp = clean_csv(file_name)
    # En caso de recibir un dataframe con datos.
    if not df_temp.empty:
      # Obtener dataframes de cantidades, montos de venta y utilidades.
      df_quantity = merge_data(df_quantity, df_temp, 'cantidad', file_name)
      
      df_sales = merge_data(df_sales, df_temp, 'precio_total', file_name)
      df_utility = merge_data(df_utility, df_temp, 'utilidad', file_name)
      

In [56]:
# Comprobación de dataframe de utilidades.
df_utility

,2021-1,2021-2,2021-3,2021-4,2021-5,2021-6,2021-7,2021-8,2021-9,2021-10,2021-11,2021-12,2022-1,2022-2,2022-3,2022-4,2022-5,2022-6,2022-7,2022-8
sku,,,,,,,,,,,,,,,,,,,,
AR0001,1097472.74,1084166.91,1075810.68,1038404.99,17431.80,20084.65,1547.64,-197739.91,-18772.37,-75144.72,-62712.20,-54593.46,22375.66,57294.40,-80623.94,7091.57,156569.73,458940.65,11552.74,-53304.54
AR0002,55323.94,54613.01,48924.57,44919.39,346.37,919.73,-429.66,18136.67,1645.49,18579.54,18204.43,9681.79,659.55,2130.48,2009.73,1073.01,3730.37,976.01,1534.25,1711.08
AR0003,110166.36,108512.95,104528.49,104064.94,482.25,1510.74,-3573.63,26852.55,511.06,26289.93,27699.17,19715.28,6939.23,5335.50,2085.88,6867.94,11167.04,8742.99,2338.82,1813.98
AR0004,161133.84,160012.68,154559.14,147480.34,1362.55,1935.82,1163.53,36746.45,1058.17,34922.98,35867.85,34705.61,6015.69,11759.63,3993.81,10403.10,22705.33,19148.51,3514.25,3357.26
AR0005,203102.32,203032.66,202621.68,200207.46,1098.28,4188.61,3638.52,-15017.75,-9688.88,-7548.25,9558.99,14672.50,7461.92,29851.77,6394.32,15233.56,31987.48,2330.32,3978.07,821.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AR3364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR3365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR3366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Comprobación de dataframe de montos.
df_sales

,2021-1,2021-2,2021-3,2021-4,2021-5,2021-6,2021-7,2021-8,2021-9,2021-10,2021-11,2021-12,2022-1,2022-2,2022-3,2022-4,2022-5,2022-6,2022-7,2022-8
sku,,,,,,,,,,,,,,,,,,,,
AR0001,2565451.19,2533017.33,2497970.82,2327202.70,52040.20,44461.77,14120.80,972524.23,17411.52,900900.81,883735.72,858042.32,66008.06,131151.12,61853.98,26423.74,557496.24,510541.34,78157.34,17159.86
AR0002,102703.56,101068.83,83333.31,73473.60,928.13,2365.11,897.79,42132.86,2163.49,37867.51,36058.33,22284.21,1014.67,3756.53,3306.14,1632.19,6012.22,2476.98,2552.85,3765.51
AR0003,216659.87,213169.12,203394.34,198668.21,3496.86,5558.79,4555.44,96998.66,2640.49,90148.80,87987.72,71897.58,9859.69,9658.44,9941.58,13819.93,21278.39,16979.98,4640.77,7036.15
AR0004,305221.36,302029.13,281943.14,262188.18,5003.61,5377.69,5522.15,136651.20,4264.10,127651.56,123379.90,118801.69,7626.95,19671.51,15793.67,21504.10,46494.15,32341.99,6552.49,9714.15
AR0005,475582.26,468180.77,457733.42,432373.73,7588.02,11239.28,6345.78,186491.89,2952.28,176555.28,172378.93,162146.22,17644.06,51533.95,29890.25,31142.60,58204.77,13666.64,12045.67,14149.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AR3364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR3365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR3366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Comprobación de dataframe de cantidades.
df_quantity

,2021-1,2021-2,2021-3,2021-4,2021-5,2021-6,2021-7,2021-8,2021-9,2021-10,2021-11,2021-12,2022-1,2022-2,2022-3,2022-4,2022-5,2022-6,2022-7,2022-8
sku,,,,,,,,,,,,,,,,,,,,
AR0001,13937.0,13722.0,13422.0,11923.0,389.0,274.0,141.0,10591.0,319.0,8910.0,8648.0,8298.0,376.0,740.0,1168.0,127.0,2403.0,2427.0,357.0,404.0
AR0002,7256.0,7130.0,5484.0,4684.0,61.0,143.0,131.0,4219.0,151.0,3613.0,3402.0,2013.0,49.0,232.0,290.0,81.0,361.0,245.0,116.0,342.0
AR0003,14109.0,13838.0,12987.0,12631.0,257.0,344.0,941.0,10335.0,260.0,9550.0,9174.0,7521.0,461.0,558.0,1120.0,897.0,1353.0,1608.0,246.0,716.0
AR0004,19878.0,19619.0,17787.0,16200.0,384.0,362.0,717.0,14317.0,417.0,13338.0,12722.0,12243.0,268.0,1215.0,1698.0,1370.0,2970.0,2885.0,322.0,823.0
AR0005,16294.0,15947.0,15472.0,14386.0,270.0,281.0,219.0,12911.0,481.0,12136.0,11138.0,10294.0,586.0,1684.0,1620.0,1032.0,1846.0,820.0,393.0,1036.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AR3364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR3365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AR3366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
# Despliegue de información del dataframe de ventas.
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3244 entries, AR0001 to AR3368
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   2021-1   3088 non-null   float64
 1   2021-2   3087 non-null   float64
 2   2021-3   3086 non-null   float64
 3   2021-4   3085 non-null   float64
 4   2021-5   1687 non-null   float64
 5   2021-6   1743 non-null   float64
 6   2021-7   1888 non-null   float64
 7   2021-8   3060 non-null   float64
 8   2021-9   1964 non-null   float64
 9   2021-10  3035 non-null   float64
 10  2021-11  3012 non-null   float64
 11  2021-12  2995 non-null   float64
 12  2022-1   2052 non-null   float64
 13  2022-2   2119 non-null   float64
 14  2022-3   2158 non-null   float64
 15  2022-4   2109 non-null   float64
 16  2022-5   2238 non-null   float64
 17  2022-6   2220 non-null   float64
 18  2022-7   2195 non-null   float64
 19  2022-8   2291 non-null   float64
dtypes: float64(20)
memory usage: 532.2+ KB


Dentro de los tres dataframes, existen valores NaN y negativos. Los NaN se deben a meses en los que algunos productos no se venden o en su defecto se acaban de agregar y no tienen histórico de ventas, por lo tanto al realizar la unión pandas asigna valores NaN a los faltantes. En el caso de los negativos, al investigar con la empresa se encontró que en ASPEL-SAE se registran todos los movimientos al inventario y por ende las devoluciones, cancelaciones, mermas y uso de producto para la propia empresa aparecen por igual generando así números negativos.



In [89]:
# Obtener valores negativos de los dataframes.
neg_quantity = (df_quantity < 0).sum().sum()
neg_sales = (df_sales < 0).sum().sum()
neg_utility = (df_utility < 0).sum().sum()

# Obtener valores NaN de los dataframes.
na_quantity = (df_quantity.isna()).sum().sum()
na_sales = (df_sales.isna()).sum().sum()
na_utility = (df_utility.isna()).sum().sum()

# Imprimir compendio de valores.
print(f'''Numeros negativos en dataframes
          Cantidades: {neg_quantity}, Ventas: {neg_sales}, Utilidades: {neg_utility}.
NAs en dataframes
          Cantidades: {na_quantity}, Ventas: {na_sales}, Utilidades: {na_utility}.''')

Numeros negativos en dataframes
          Cantidades: 175, Ventas: 266, Utilidades: 4025.
NAs en dataframes
          Cantidades: 15768, Ventas: 15768, Utilidades: 15768.


Aunque para el caso de las utilidades existe el caso en que se tengan pérdidas (saldos negativos) debido a que el producto no tuvo buen comportamiento o alguna eventualidad y por algún motivo se tuvo que vender más barato, para efectos de este análisis se sustituyen por ceros los valores negativos así como los valores NaN ya que representan que el producto no ha tenido movimientos.

In [93]:
# Reemplazar negativos por cero.
df_quantity_clean = df_quantity
df_quantity_clean[df_quantity_clean < 0] = 0
df_sales_clean = df_sales
df_sales_clean[df_sales_clean < 0] = 0
df_utility_clean = df_utility
df_utility_clean[df_utility_clean < 0] = 0

# Remplazar NaNs por cero.
df_quantity_clean = df_quantity.fillna(0)
df_sales_clean = df_sales.fillna(0)
df_utility_clean = df_utility.fillna(0)

# Obtener valores negativos y NaN de los dataframes.
neg_quantity = (df_quantity_clean < 0).sum().sum()
neg_sales = (df_sales_clean < 0).sum().sum()
neg_utility = (df_utility_clean < 0).sum().sum()
na_quantity = (df_quantity_clean.isna()).sum().sum()
na_sales = (df_sales_clean.isna()).sum().sum()
na_utility = (df_utility_clean.isna()).sum().sum()

# Imprimir compendio de valores.
print(f'''Numeros negativos en dataframes
          Cantidades: {neg_quantity}, Ventas: {neg_sales}, Utilidades: {neg_utility}.
NAs en dataframes
          Cantidades: {na_quantity}, Ventas: {na_sales}, Utilidades: {na_utility}.''')

Numeros negativos en dataframes
          Cantidades: 0, Ventas: 0, Utilidades: 0.
NAs en dataframes
          Cantidades: 0, Ventas: 0, Utilidades: 0.


Para estandarizar los nombres de las columnas se renombran siguiendo la nomenclatura 'mes_año' utilizando un mnemónico de 3 caracteres para el mes.

In [97]:
# Lista de nombres de columnas estandarizadas.
column_names = ['ene_21', 'feb_21', 'mar_21', 'abr_21', 'may_21', 'jun_21',  
                'jul_21', 'ago_21', 'sep_21', 'oct_21', 'nov_21', 'dic_21',
                'ene_22', 'feb_22', 'mar_22', 'abr_22', 'may_22', 'jun_22',  
                'jul_22', 'ago_22']

# Renombramiento de columnas en dataframes.
df_quantity_clean.columns = column_names
df_sales_clean.columns = column_names
df_utility_clean.columns = column_names

# Mostrar cambios con un dataframe de muestra
df_sales_clean

,ene_21,feb_21,mar_21,abr_21,may_21,jun_21,jul_21,ago_21,sep_21,oct_21,nov_21,dic_21,ene_22,feb_22,mar_22,abr_22,may_22,jun_22,jul_22,ago_22
sku,,,,,,,,,,,,,,,,,,,,
AR0001,2565451.19,2533017.33,2497970.82,2327202.70,52040.20,44461.77,14120.80,972524.23,17411.52,900900.81,883735.72,858042.32,66008.06,131151.12,61853.98,26423.74,557496.24,510541.34,78157.34,17159.86
AR0002,102703.56,101068.83,83333.31,73473.60,928.13,2365.11,897.79,42132.86,2163.49,37867.51,36058.33,22284.21,1014.67,3756.53,3306.14,1632.19,6012.22,2476.98,2552.85,3765.51
AR0003,216659.87,213169.12,203394.34,198668.21,3496.86,5558.79,4555.44,96998.66,2640.49,90148.80,87987.72,71897.58,9859.69,9658.44,9941.58,13819.93,21278.39,16979.98,4640.77,7036.15
AR0004,305221.36,302029.13,281943.14,262188.18,5003.61,5377.69,5522.15,136651.20,4264.10,127651.56,123379.90,118801.69,7626.95,19671.51,15793.67,21504.10,46494.15,32341.99,6552.49,9714.15
AR0005,475582.26,468180.77,457733.42,432373.73,7588.02,11239.28,6345.78,186491.89,2952.28,176555.28,172378.93,162146.22,17644.06,51533.95,29890.25,31142.60,58204.77,13666.64,12045.67,14149.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AR3364,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AR3365,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
AR3366,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


En el caso del dataframe de cantidades el tipo de dato cambió de enteros a flotantes por lo que se le hace un casting para regresarlo a enteros. Los otros dos dataframes conservan su tipo de variable.

In [99]:
df_quantity_clean = df_quantity_clean.astype('int')
df_quantity_clean.head(10)

,ene_21,feb_21,mar_21,abr_21,may_21,jun_21,jul_21,ago_21,sep_21,oct_21,nov_21,dic_21,ene_22,feb_22,mar_22,abr_22,may_22,jun_22,jul_22,ago_22
sku,,,,,,,,,,,,,,,,,,,,
AR0001,13937,13722,13422,11923,389,274,141,10591,319,8910,8648,8298,376,740,1168,127,2403,2427,357,404
AR0002,7256,7130,5484,4684,61,143,131,4219,151,3613,3402,2013,49,232,290,81,361,245,116,342
AR0003,14109,13838,12987,12631,257,344,941,10335,260,9550,9174,7521,461,558,1120,897,1353,1608,246,716
AR0004,19878,19619,17787,16200,384,362,717,14317,417,13338,12722,12243,268,1215,1698,1370,2970,2885,322,823
AR0005,16294,15947,15472,14386,270,281,219,12911,481,12136,11138,10294,586,1684,1620,1032,1846,820,393,1036
AR0006,2777,2601,2554,2386,235,214,268,1340,80,1098,882,840,119,37,93,94,96,95,25,114
AR0007,5452,5231,4997,4542,512,227,107,3441,501,2644,2365,2301,397,424,176,35,156,342,176,330
AR0008,16019,15756,15411,14810,504,337,322,12033,529,9953,9514,8871,382,674,354,2406,715,2495,700,301
AR0009,97,96,96,85,10,5,0,70,7,62,60,52,1,0,14,1,10,5,15,6


Finalmente se guardan los nuevos dataframes en archivos CSV para su posterior manenejo.

In [100]:
# Escritura a un CSV del dataframe limpio.
df_quantity_clean.to_csv('quantity_clean.csv')
df_sales_clean.to_csv('sales_clean.csv')
df_utility_clean.to_csv('utility_clean.csv')